In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [2]:
def split_string_ref(str_fullref: str):
    collection, key = str_fullref.split("/")
    return collection, key

# Carrega os dados que já estão salvos

In [3]:
items = load_json_file("json1123/itens.json")
volumes = load_json_file("json1123/volumes.json")

In [4]:
def load_data_from_fullref(fullref: str):
    collection, key = split_string_ref(fullref)
    if collection == "volumes":  
        return volumes[key]
    elif collection == "items":
        return items[key]
    

# Nossos itens que já foram mensurados

In [18]:
itens_mensurados = list(filter(lambda x: re.match(r".*\(AxCxL\)", x["detalhes"]["medidas"]) != None, items.values()))
itens_mensurados_keys = [x["key"] for x in itens_mensurados]

# Regra 1

Vamos procurar por mobília que tem mais de 1 item

In [9]:
mobilia_errada = list(filter(lambda x: x["categoria"] == "Mobília" and len(x["items"]) > 1 and x["origem"] !=  "ambientes/A001", volumes.values())) 

In [10]:
mobilia_errada_codigo = [x["codigo"] for x in mobilia_errada]

In [11]:
# filtra os itens que estão volumados em algum volume que está na lista de mobilia errada
itens_mensurados_volumados_errados = list(filter(lambda x: x["meta"]["volume"] != None and x["meta"]["volume"].split("/")[1] in mobilia_errada_codigo, itens_mensurados))

## Teste 1

In [12]:
def extract_medidas_loose(descricao: str):
    """
    Extracts the measures from a string
    :param descricao: string to extract the measures from
    :return: a tuple of integers with the extracted measures
    """ 
    return tuple(map(int, re.findall(r'[xX]*(\d+).*?[Xx]', descricao)))

def extract_medidas(item: dict) -> dict:
    medidas = item["detalhes"]["medidas"]
    match = re.search(r"(\d+) x (\d+) x (\d+)", medidas)
    if match == None: 
        raise Exception(f"Não foi possível extrair as medidas do item {item['key']}")
    else:
        medidas_groups = match.groups()
        medidas_int = list(map(lambda x: int(x), medidas_groups))
        return dict(zip(["a", "c", "l"], medidas_int)) 

In [54]:
volume_teste = volumes['cute3qhONPhHp3D0rWlh']

In [13]:


def clone_original_volume(item: dict) -> dict:
    """Recebe um item e retorna um volume com os dados do item e o item como único item do volume"""
    volume_fullref = item["meta"]["volume"]
    volume_copy = load_data_from_fullref(volume_fullref).copy()
    volume_copy["items"] = [ db.document(f"items/{item['key']}") ]
    for prop in ["responsavel", "origem", "destino", "localizacao_atual"]:
        volume_copy[prop] = db.document(volume_copy[prop])
    volume_copy["data_criacao"] = datetime.now()
    volume_copy["codigo"] = f"V{item['key']}"
    volume_copy["medidas"] = extract_medidas(item)
    volume_copy["observacao"] = "Criado pelo sistema. Volume original: " + volume_fullref
    return volume_copy

def clone_original_volume_error(item: dict) -> dict:
    """Recebe um item e retorna um volume com os dados do item e o item como único item do volume"""
    volume_fullref = item["meta"]["volume"]
    volume_copy = load_data_from_fullref(volume_fullref).copy()
    volume_copy["items"] = [ db.document(f"items/{item['key']}") ]
    for prop in ["responsavel", "origem", "destino", "localizacao_atual"]:
        volume_copy[prop] = db.document(volume_copy[prop])
    volume_copy["data_criacao"] = datetime.now()
    volume_copy["codigo"] = f"V{item['key']}"
    volume_copy["medidas"] =  { "a": 1, "c": 2, "l": 3}
    volume_copy["observacao"] = "Criado pelo sistema. PRECISA AJUSTAR AS MEDIDAS. Volume original: " + volume_fullref 
    return volume_copy

def soft_delete_volume(volume_fullref: str):
    volume_db = db.document(volume_fullref)
    volume_db.update({"deleted": True})
    

In [14]:
NEW_VOLUMES = []

output = {'certos': 0, 'errados': 0}
for mobilia in mobilia_errada:
    errors = 0
    volume_fullref = f"volumes/{mobilia['codigo']}"
    print(f"Processando o volume {volume_fullref}")
    for item in mobilia["items"]:
        item_dict = load_data_from_fullref(item)
        new_volume = None
        if isinstance(item_dict["key"],int):
            print(f"\t{bcolors.OKBLUE}O item {item_dict['key']} é patrimoniado.{bcolors.ENDC}")
        else:
            print(f"\t{bcolors.WARNING}O item {item_dict['key']} não é patrimoniado.{bcolors.ENDC}")
        try:
            new_volume = clone_original_volume(item_dict)
            print(f"\t\t{bcolors.OKGREEN}O item {item_dict['key']} foi volumado com sucesso.{bcolors.ENDC}")
            output['certos'] += 1
        except Exception as e:
            output['errados'] += 1
            errors += 1
            new_volume = clone_original_volume_error(item_dict)
            print(f"\t\t{bcolors.FAIL}O item {item_dict['key']} foi volumado com erros: não foi possível determinar suas medidas.{bcolors.ENDC}")
        NEW_VOLUMES.append(new_volume)
    if errors == 0:
        print(f"{bcolors.OKGREEN}Tudo certo com o volume {volume_fullref}{bcolors.ENDC}")
    else:
        print(f"{bcolors.FAIL}O volume {volume_fullref} tem {errors} erros{bcolors.ENDC}")

Processando o volume volumes/0AoXSY5VHYKD71VYYOlL
	O item 337879 é patrimoniado.
		O item 337879 foi volumado com sucesso.
	O item 337880 é patrimoniado.
		O item 337880 foi volumado com sucesso.
	O item 337881 é patrimoniado.
		O item 337881 foi volumado com sucesso.
	O item 337883 é patrimoniado.
		O item 337883 foi volumado com sucesso.
Tudo certo com o volume volumes/0AoXSY5VHYKD71VYYOlL
Processando o volume volumes/0fQ6fsbYJ57tNzqRhSSN
	O item 327227 é patrimoniado.
		O item 327227 foi volumado com sucesso.
	O item 327486 é patrimoniado.
		O item 327486 foi volumado com sucesso.
	O item 338175 é patrimoniado.
		O item 338175 foi volumado com sucesso.
	O item 338174 é patrimoniado.
		O item 338174 foi volumado com sucesso.
Tudo certo com o volume volumes/0fQ6fsbYJ57tNzqRhSSN
Processando o volume volumes/1SCTls7dZYxuBIhylcSM
	O item 347946 é patrimoniado.
		O item 347946 foi volumado com sucesso.
	O item 347947 é patrimoniado.
		O item 347947 foi volumado com sucesso.
	O item 347948

In [20]:
def batch_write(collection: str, dictarray: list[dict], key_id: str | None = None):
    batch = db.batch()
    collection_ref = db.collection(collection)
    k = 1
    for i in range(0, len(dictarray), 500):
        chunk = dictarray[i:i+500]
        for document in chunk:
            if key_id:
                doc_ref = collection_ref.document(str(document[key_id]))
                batch.set(doc_ref, document)
        print(f"{collection}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [21]:
batch_write("volumes", NEW_VOLUMES, "codigo")

volumes: chunk 1 (500 documents)
volumes: chunk 2 (496 documents)


# Regra 2

In [28]:
escritorio_preerrado = list(filter(lambda x: x["categoria"] == "Material de Escritório" and len(x["items"]) > 1, volumes.values())) 

In [29]:
items_escritorio_preerrado = [ [ load_data_from_fullref(x) for x in volume["items"] ] for volume in escritorio_preerrado ]

In [24]:
from functools import reduce

In [31]:
filtro_items_escrits = [ reduce(lambda x,y: x or y, map(lambda x: x['key'] in itens_mensurados_keys, x))  for x in items_escritorio_preerrado ] 

In [32]:
for mask, volume in zip(filtro_items_escrits, escritorio_preerrado):
    if mask:
        print(f"{bcolors.WARNING}O volume {volume['codigo']} tem itens mensurados{bcolors.ENDC}")
    else:
        print(f"{bcolors.OKGREEN}O volume {volume['codigo']} não tem itens mensurados{bcolors.ENDC}")

O volume C308000A não tem itens mensurados
O volume C3010002 tem itens mensurados
O volume C3060007 tem itens mensurados
O volume C3010007 tem itens mensurados
O volume C3030004 tem itens mensurados
O volume C3050004 tem itens mensurados
O volume C3050003 tem itens mensurados
O volume C3020003 tem itens mensurados
O volume B0120012 não tem itens mensurados
O volume C2040007 tem itens mensurados
O volume C3050005 tem itens mensurados
O volume C3020005 tem itens mensurados
O volume C3060006 tem itens mensurados
O volume C3010005 tem itens mensurados
O volume C3050001 tem itens mensurados
O volume C3060003 tem itens mensurados
O volume C3020002 tem itens mensurados
O volume C3040008 tem itens mensurados
O volume C3080007 não tem itens mensurados
O volume C3030001 tem itens mensurados
O volume C3040003 tem itens mensurados
O volume C3060004 tem itens mensurados
O volume A00100DD não tem itens mensurados
O volume C2040002 tem itens mensurados
O volume C3050002 tem itens mensurados
O volume 